In [3]:
# Just run this block. Please do not modify the following code.
import math
import time
import io
import numpy as np
import csv
from IPython.display import Image

# Pytorch package
import torch
import torch.nn as nn
import torch.optim as optim

# Torchtest package
import torchtext
from torchtext.datasets import Multi30k
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab
from torchtext.utils import download_from_url, extract_archive
from torch.nn.utils.rnn import pad_sequence

# Tqdm progress bar
from tqdm import tqdm_notebook, tqdm

# Code provide to you for training and evaluation

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

/opt/homebrew/anaconda3/envs/cs7643-finalproject/lib/python3.9/site-packages/torchtext/datasets/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/opt/homebrew/anaconda3/envs/cs7643-finalproject/lib/python3.9/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/opt/homebrew/anaconda3/envs/cs7643-finalproject/lib/python3.9/site-pack

In [10]:
encoder_output = torch.rand((5, 320, 1)) # N, input_size, 1

In [32]:
from Decoder import Decoder
from PostNet import PostNet

In [33]:
decoder = Decoder()
decoder_output = decoder.forward(encoder_output)
print(decoder_output.shape) # N, output_size, 1
print(decoder_output)

torch.Size([5, 80, 1])
tensor([[[-0.0041],
         [ 0.0065],
         [-0.0150],
         [-0.0346],
         [-0.0223],
         [-0.0142],
         [-0.0367],
         [ 0.0065],
         [-0.0144],
         [ 0.0167],
         [ 0.0091],
         [-0.0064],
         [ 0.0246],
         [ 0.0005],
         [-0.0289],
         [ 0.0180],
         [-0.0152],
         [-0.0260],
         [-0.0167],
         [ 0.0278],
         [-0.0247],
         [-0.0231],
         [ 0.0320],
         [-0.0010],
         [ 0.0232],
         [ 0.0162],
         [-0.0005],
         [ 0.0142],
         [-0.0148],
         [-0.0297],
         [ 0.0072],
         [-0.0192],
         [ 0.0210],
         [ 0.0274],
         [ 0.0287],
         [-0.0182],
         [-0.0337],
         [-0.0028],
         [-0.0239],
         [-0.0277],
         [ 0.0244],
         [ 0.0239],
         [ 0.0093],
         [ 0.0154],
         [ 0.0227],
         [-0.0317],
         [ 0.0068],
         [-0.0090],
         [ 0.0266

In [34]:
postnet = PostNet()
postnet_output = postnet.forward(decoder_output)
print(postnet_output.shape) # (N, output size, 1)
print(postnet_output)

torch.Size([5, 80, 1])
tensor([[[ 0.4608],
         [ 0.2008],
         [ 0.3139],
         [-0.1172],
         [-0.0809],
         [-0.2830],
         [-0.2359],
         [-0.0952],
         [ 0.0126],
         [ 0.0107],
         [ 0.1856],
         [-0.1084],
         [ 0.0426],
         [-0.0495],
         [-0.2653],
         [ 0.2008],
         [-0.1558],
         [-0.2693],
         [ 0.0314],
         [-0.4720],
         [-0.0179],
         [-0.1181],
         [ 0.1171],
         [ 0.0694],
         [ 0.1522],
         [ 0.0157],
         [-0.2195],
         [-0.1613],
         [-0.2345],
         [ 0.1223],
         [ 0.3862],
         [-0.1988],
         [-0.2089],
         [-0.0820],
         [-0.0296],
         [-0.2763],
         [-0.0959],
         [ 0.1711],
         [-0.1045],
         [-0.0823],
         [-0.0936],
         [ 0.1615],
         [ 0.1119],
         [-0.2503],
         [-0.0249],
         [-0.1114],
         [ 0.0101],
         [-0.1204],
         [-0.0690

In [36]:
# Loss Test
from utils import criterion

input = torch.rand((5, 80, 1))

criterion(input=input, encoder_ouput=encoder_output, decoder_output=decoder_output, postnet_output=postnet_output, encoded_postnet_output=encoder_output)

tensor(0.6855, grad_fn=<AddBackward0>)